In [21]:
# this code implements a deep neural network for feature learning
# this code comes from Vishwa's class on constructing a data loader and creating a model

# Some common system imports
import os
import sys
import importlib
import time
import csv

# Numeric computing
import numpy as np

# Sklearn functions are useful for generating train/test splits, and metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

from scipy.io import wavfile

# pytorch
import torch
import torch.utils.data as tdata
import torchaudio
from torch.nn.functional import pad
from torchaudio import transforms
from torchaudio.sox_effects import apply_effects_tensor

# Plotting (if we want it)
import matplotlib.pyplot as plt

# importing our own modules
import audio_datasets as ads

In [22]:
# loading the raw data for feature learning
training_path = os.path.join(os.getcwd(), "..", "training_data") #need to change this back before pushing ***
files = os.listdir(training_path)
wav_files = []
file_type = "wav"
LABELS = {"neutral": 0, "calm": 1, "happy": 2, "sad": 3, "angry": 4, "fearful": 5, "disgust": 6, "surprised": 7}

for file in files:
    curr_path = os.path.join(training_path, file)
    if os.path.isfile(curr_path) and file_type in file:
        for label in LABELS.keys():
            if label in file:
                wav_files.append((file, LABELS[label]))

data_array = []
label_array = []

for data in wav_files: #in the my_datasets code
    waveform, samplerate = torchaudio.load(os.path.join(training_path, data[0]))
    voice_detect = transforms.Vad(samplerate) # Consider adjusting some parameters
    front_trimmed_waveform = voice_detect(waveform)
    reversed_trimmed_waveform, _ = apply_effects_tensor(front_trimmed_waveform, samplerate, [["reverse"]])
    reversed_full_trimmed_waveform = voice_detect(reversed_trimmed_waveform)
    full_trimmed_waveform, _ = apply_effects_tensor(reversed_full_trimmed_waveform, samplerate, [["reverse"]])
    data_array.append(full_trimmed_waveform)
    label_array.append(data[1])

max_length = max([data.shape[1] for data in data_array])
train_data_array = [pad(data, [1, max_length]) for data in data_array]

In [23]:
# loading the raw data for feature learning
test_path = os.path.join(os.getcwd(), "..", "test_data")
test_files = os.listdir(test_path)
test_wav_files = []

for file in test_files:
    curr_path = os.path.join(test_path, file)
    if os.path.isfile(curr_path) and file_type in file:
        test_wav_files.append(file)

test_data_array = []
for data in test_wav_files: #in the my_datasets code
    waveform, samplerate = torchaudio.load(os.path.join(test_path, data))
    voice_detect = transforms.Vad(samplerate) # Consider adjusting some parameters
    front_trimmed_waveform = voice_detect(waveform)
    reversed_trimmed_waveform, _ = apply_effects_tensor(front_trimmed_waveform, samplerate, [["reverse"]])
    reversed_full_trimmed_waveform = voice_detect(reversed_trimmed_waveform)
    full_trimmed_waveform, _ = apply_effects_tensor(reversed_full_trimmed_waveform, samplerate, [["reverse"]])
    test_data_array.append(full_trimmed_waveform)

test_data_array = [pad(data, [1, max_length]) for data in test_data_array]

In [27]:
max_length = max([data.shape[1] for data in train_data_array])
print("Prethingy", max_length)
train_data_array = [pad(data, [1, max_length]) for data in train_data_array]
max_length = max([data.shape[1] for data in train_data_array])
print(max_length)
train_ten = torch.stack(train_data_array)
test_ten = torch.stack(test_data_array)
train_y_ten = torch.tensor(label_array)

360433


RuntimeError: stack expects each tensor to be equal size, but got [1, 231579] at entry 0 and [1, 233017] at entry 1

In [9]:
# #implementing the Mel-frequency cepstrum coefficients as feature vectors
# mfcc = transforms.MFCC(sample_rate=48000, n_mfcc=60)
# print(train_ten.shape)
# #mel_spectrogram = transforms.MelSpectrogram(sample_rate=48000, n_fft=400, n_mels=1)
# #print("2", mel_spectrogram(train_ten[0].float()).shape)
# #print("2.5", mfcc(mel_spectrogram(train_ten[0].float())).shape)
# print(mfcc(train_ten[0].float()).shape)
# mfcc_features = np.zeros((len(label_array), 72060))
# for row in range(len(label_array)):
#     mel_coef = np.array(mfcc(train_ten[row].float()))
#     mel_coef = mel_coef.reshape((1, 72060))
#     mfcc_features[row] = mel_coef

# print("3", train_ten.size())
# print("4", mfcc_features.shape)
# print("5", train_y_ten.shape)



/home/ethandp/venvs/301-env/lib/python3.8/site-packages/torchaudio/functional/functional.py:571: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


torch.Size([1125, 240000])
torch.Size([60, 1201])
3 torch.Size([1125, 240000])
4 (1125, 72060)
5 torch.Size([1125])


In [11]:
#implementing the Mel-frequency cepstrum coefficients as feature vectors
mfcc2 = transforms.MFCC(sample_rate=48000, n_mfcc=60)
#mel_spectrogram = transforms.MelSpectrogram(sample_rate=48000, n_fft=400, n_mels=1)
#print("2", mel_spectrogram(train_ten[0].float()).shape)
#print("2.5", mfcc(mel_spectrogram(train_ten[0].float())).shape)
mfcc_test_features = np.zeros((len(test_data_array), 72060))
for row in range(len(test_data_array)):
    mel_coef = np.array(mfcc2(test_ten[row].float()))
    mel_coef = mel_coef.reshape((1, 72060))
    mfcc_test_features[row] = mel_coef


/home/ethandp/venvs/301-env/lib/python3.8/site-packages/torchaudio/functional/functional.py:571: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [16]:
forest2 = RandomForestClassifier(n_estimators=10000) #gnb and y_pred lines directly from scikit-learn website documentation for Naive-Bayes
print(mfcc_features.shape, train_y_ten.shape)
y_pred2 = forest2.fit(mfcc_features, label_array).predict(mfcc_test_features)
y_pred2 = list(y_pred2)

#print(classification_report(test_y_ten, test_pred))
# correct = 0
# for label in range(len(y_pred2)):
#     if y_pred2[label] == test_labels[label]:
#         correct += 1
# print("Number of correct labels: ", correct)

LABEL_CONVERTER = dict([(value, key) for key, value in LABELS.items()])

#counting the mislabeled points
with open("random_forest_results2.csv", "w+") as result_file:
   writer = csv.writer(result_file)
   writer.writerow(["filename", "label"])

   for idx in range(len(y_pred2)):
       writer.writerow([os.path.splitext(test_wav_files[idx])[0], LABEL_CONVERTER[y_pred2[idx]]])
#print("Number of mislabeled points out of a total %d points : %d", (len(test_labels), (test_labels != y_pred2).sum()))

(1125, 72060) torch.Size([1125])
